In [1]:
import pandas as pd


In [2]:
amz = pd.read_csv('MLOpsReviews/amazon_prime_titles.csv')
dnp = pd.read_csv('MLOpsReviews/disney_plus_titles.csv')
hulu = pd.read_csv('MLOpsReviews/hulu_titles.csv')
ntx = pd.read_csv('MLOpsReviews/netflix_titles.csv')

In [3]:
def plataform(data,plataform):
  data['plataform'] = plataform
  return data

def duration_int(row,cont):
  try:
    if isinstance(row.duration,str):
      aux = row.duration.split(' ')
      if not aux[cont].isnumeric() and cont == 0:
        return None
      else:
        return aux[cont]
    else:
      return None
  except IndexError:
    return None

In [4]:
amz= plataform(amz,'amazon prime')
dnp = plataform(dnp,'disney plus')
hulu = plataform(hulu,'hulu')
ntx =plataform(ntx,'netflix')

In [5]:
data = pd.concat([amz,dnp,hulu,ntx]) # creacion de un unico dataset

In [6]:
data = data.reset_index(drop=True) # reseteo de index

In [7]:
data.show_id = data.apply(lambda row : row['plataform'][0] + row.show_id, axis=1) # cambio de id

In [8]:
data.date_added=pd.to_datetime(data.date_added) # cambio formato de fecha
data.loc[pd.notnull(data.date_added),'date_added']= pd.to_datetime(data.loc[pd.notnull(data.date_added),'date_added']).dt.strftime('%Y-%m-%d')

In [ ]:
for row , dat in data.iterrows():
    if isinstance(dat.rating,str):
        if 'min' in dat.rating or 'Season' in dat.rating or 'Seasons' in dat.rating: 
          data.duration.loc[row] = dat.rating
          data.rating.loc[row] = 'G'

In [10]:
data.rating = data.rating.fillna('G') # elimina todos los Nan de rating

In [11]:
data["duration_int"] = data.apply(lambda row: duration_int(row,0),axis=1)
data["duration_type"] = data.apply(lambda row: duration_int(row,1),axis=1)

In [12]:
data = data.drop(columns=['duration'],axis=1)

In [13]:
cambio = {'Seasons':'season',
            'min':'min',
            'Season' : 'season'}

In [14]:
data.duration_type = data.duration_type.map(cambio)

In [15]:
data = data.applymap(lambda x:x.lower() if type(x) == str else x) #pasar el dataframe a minuscula

In [16]:
change_code= {'as943':'as942', 'as5894':'as5087', 'as9037': 'as4402', 'as9059':'as7296', 'as9270' :'as7054', 'ns6706':'ns304','ns7346':'ns160', 'ns8023':'ns1271'}

In [25]:
data.show_id = data.show_id.replace(change_code)

In [26]:
data= data.drop_duplicates(subset=['title','release_year','plataform','duration_int'])

In [17]:
df_rating = pd.read_csv('rating.csv')

In [29]:
df_rating.movieId = df_rating.movieId.replace(change_code)

In [18]:
df_rating.columns

Index(['Unnamed: 0', 'userId', 'rating', 'timestamp', 'movieId'], dtype='object')

In [34]:
grup_movie = df_rating.groupby('movieId').agg({'rating':'mean'}).round()

In [36]:
grup_movie.shape

(22990, 1)

## Se crea una dataset nuevo con los score(promedio de las calificaciones por cada pelicula)  

In [37]:
dataset= pd.merge(data, grup_movie ,left_on='show_id',right_on='movieId')

In [39]:
dataset.rename(columns={'rating_y':'score'},inplace=True)

In [40]:
#dataset.to_csv('movie_clean.csv',index=False)